In [1]:
!wget https://huggingface.co/garyw/clinical-embeddings-100d-w2v-cr/resolve/main/w2v_OA_CR_100d.bin
!wget https://huggingface.co/garyw/clinical-embeddings-100d-w2v-cr/resolve/main/w2v_OA_CR_100d.bin.wv.vectors.npy
!wget https://huggingface.co/garyw/clinical-embeddings-100d-w2v-cr/resolve/main/w2v_OA_CR_100d.bin.trainables.syn1neg.npy
!pip install fastparquet

--2024-06-19 14:36:49--  https://huggingface.co/garyw/clinical-embeddings-100d-w2v-cr/resolve/main/w2v_OA_CR_100d.bin
Resolving huggingface.co (huggingface.co)... 65.8.243.46, 65.8.243.90, 65.8.243.92, ...
Connecting to huggingface.co (huggingface.co)|65.8.243.46|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/e8/72/e872abef7b2278b5cf6a4f41d1455f5d309cdbd6310b2da0a6e9aecc840217e7/7f920b29a2c3861a475f296d397ff679528b064dbb51debf33568ae5f4f7e088?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27w2v_OA_CR_100d.bin%3B+filename%3D%22w2v_OA_CR_100d.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1719067009&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxOTA2NzAwOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9lOC83Mi9lODcyYWJlZjdiMjI3OGI1Y2Y2YTRmNDFkMTQ1NWY1ZDMwOWNkYmQ2MzEwYjJkYTBhNmU5YWVjYzg0MDIxN2U3LzdmOTIwYjI5YTJjMzg2MWE0NzVmMj

In [40]:
from gensim.models import FastText, Word2Vec, KeyedVectors
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from nltk.stem import WordNetLemmatizer
import string
import nltk
from torch.utils.data import Dataset,DataLoader, random_split
import re
from numpy import random
from sklearn.metrics import accuracy_score
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
model = Word2Vec.load('w2v_OA_CR_100d.bin')
print(model.wv.get_vector('lymphangioleiomyomatosis'))

[-0.6044092   0.4017609  -0.717026   -0.2701869  -0.23817156 -0.34242344
 -0.15332928 -0.11580862 -0.36377856 -0.10749034  0.4498769   0.7072215
 -0.4689228  -0.48433578  1.050483    0.71655744 -0.6439546  -0.44996452
 -0.11657333  0.09194979  0.35634688  0.4884644   0.28647774 -0.496315
 -0.7018925  -0.25540805  0.14061369 -0.8933765  -0.52663106  0.40501425
  0.17843099  0.21091795 -0.231396   -0.18487869  0.38206643  0.09275728
 -0.4573524   0.7668036   0.17861798  0.31376782 -0.49365893 -0.5377006
  0.27592292 -0.04515044 -0.23638678 -0.5425362  -0.46474802  0.10646814
 -0.17016436 -0.07704978  0.17462458  0.27287003  0.5647319   0.1521702
  0.1517999   0.2716373   0.21111147 -0.11035519  0.19445771  0.14426446
  0.7005133  -0.71069545  0.01735174  0.30701727 -0.54055333 -0.0802884
 -0.1946127  -0.4616384  -0.4518422  -0.00218993 -0.3471012   0.60170263
 -0.47892392 -0.04716599  0.07105272  0.35743713 -0.4405513  -0.53299844
 -0.21108003  0.10141873  0.25599658 -0.22999842  0.21882

In [4]:
test_data = pd.read_parquet('test-00000-of-00001-47685aa42db61e77.parquet', engine='fastparquet')
train_data = pd.read_parquet('train-00000-of-00001-210cfe9263b99806.parquet', engine='fastparquet')
valid_data = pd.read_parquet('valid-00000-of-00001-cc552de6d1a6fa4b.parquet', engine='fastparquet')
test_data

,id,query,answer,choices,gold
0,MedNLI0,\nTASK: Please classify the relationship betwe...,entailment,"[entailment, contradiction, neutral]",0
1,MedNLI1,\nTASK: Please classify the relationship betwe...,contradiction,"[entailment, contradiction, neutral]",1
2,MedNLI2,\nTASK: Please classify the relationship betwe...,neutral,"[entailment, contradiction, neutral]",2
3,MedNLI3,\nTASK: Please classify the relationship betwe...,entailment,"[entailment, contradiction, neutral]",0
4,MedNLI4,\nTASK: Please classify the relationship betwe...,contradiction,"[entailment, contradiction, neutral]",1
...,...,...,...,...,...
1417,MedNLI1417,\nTASK: Please classify the relationship betwe...,contradiction,"[entailment, contradiction, neutral]",1
1418,MedNLI1418,\nTASK: Please classify the relationship betwe...,neutral,"[entailment, contradiction, neutral]",2
1419,MedNLI1419,\nTASK: Please classify the relationship betwe...,entailment,"[entailment, contradiction, neutral]",0
1420,MedNLI1420,\nTASK: Please classify the relationship betwe...,contradiction,"[entailment, contradiction, neutral]",1


In [5]:
def find_pre_and_hyp(query):
    start_pre = query.find("[PRE]") + len("[PRE]")
    end_pre = query.find("[HYP]")
    start_hyp = query.find("[HYP]") + len("[HYP]")
    end_hyp = query.find("OUTPUT:")
    premise = query[start_pre:end_pre].strip()
    hypothesis = query[start_hyp:end_hyp].strip()

    return premise,hypothesis

In [7]:
def get_texts_tokens(data):
    premise_texts_tokens = []
    hypothesis_texts_tokens = []
    lemmatizer = WordNetLemmatizer()
    for query,answer in zip(data['query'],data['answer']):
        premise,hypothesis = find_pre_and_hyp(query)
        premise = premise.replace('-','')
        hypothesis = hypothesis.replace('-','')
        hypothesis = 'start ' + hypothesis
        tokens_premise = nltk.word_tokenize(premise)
        tokens_hypothesis = nltk.word_tokenize(hypothesis)
        tokens_premise = [lemmatizer.lemmatize(token.lower()) for token in tokens_premise if token not in string.punctuation]
        tokens_hypothesis = [lemmatizer.lemmatize(token.lower()) for token in tokens_hypothesis if token not in string.punctuation]
        premise_texts_tokens.append(tokens_premise)
        hypothesis_texts_tokens.append(tokens_hypothesis)
    return premise_texts_tokens,hypothesis_texts_tokens

In [9]:
train_premise_texts_tokens,train_hypothesis_texts_tokens = get_texts_tokens(train_data)
test_premise_texts_tokens,test_hypothesis_texts_tokens = get_texts_tokens(test_data)

In [10]:
def get_vectors_of_given_word(word):
    try:
       return model.wv.get_vector(word)
    except:
       return random.uniform(-1,1,size=(100,))
       #return None

In [16]:
def get_x_premise_arr(premise_texts_tokens):
    x_premise_arr = []
    for text in premise_texts_tokens:
        x = np.empty((0,100))
        for word in text:
            vectors = get_vectors_of_given_word(word)
            vectors_np = np.array(vectors)
            x = np.vstack([x, vectors_np])
        x_premise_arr.append(x)
    return x_premise_arr

In [17]:
train_x_premise_arr = get_x_premise_arr(train_premise_texts_tokens)
test_x_premise_arr = get_x_premise_arr(test_premise_texts_tokens)

In [18]:
def get_x_hypothesis_arr(hypothesis_texts_tokens):
    x_hypothesis_arr = []
    for text in hypothesis_texts_tokens:
        x = np.empty((0,100))
        for word in text:
            vectors = get_vectors_of_given_word(word)
            vectors_np = np.array(vectors)
            x = np.vstack([x, vectors_np])
        x_hypothesis_arr.append(x)
    return x_hypothesis_arr

In [19]:
train_x_hypothesis_arr = get_x_hypothesis_arr(train_hypothesis_texts_tokens)
test_x_hypothesis_arr = get_x_hypothesis_arr(test_hypothesis_texts_tokens)

In [26]:
def get_y_arr(data):
    y_arr = []
    for query,answer in zip(data['query'],data['answer']):
        if answer == 'entailment':
           y = [1,0,0]
        elif answer == 'neutral':
           y = [0,1,0]
        elif answer == 'contradiction':
           y = [0,0,1]
        else:
           print('should not get here')
        y_arr.append(y)
    return y_arr

In [27]:
train_y_arr = get_y_arr(train_data)
test_y_arr = get_y_arr(test_data)

In [28]:
for x_premise in train_x_premise_arr:
    if x_premise.shape[0] == 0:
       print(x_premise.shape)
for x_premise in test_x_premise_arr:
    if x_premise.shape[0] == 0:
       print(x_premise.shape)

In [29]:
for x_hypothesis in train_x_hypothesis_arr:
    if x_hypothesis.shape[0] == 0:
       print(x_hypothesis.shape)
for x_hypothesis in test_x_hypothesis_arr:
    if x_hypothesis.shape[0] == 0:
       print(x_hypothesis.shape)

In [30]:
print(len(train_x_hypothesis_arr))
print(len(train_x_premise_arr))
print(len(train_y_arr))

11232
11232
11232


In [31]:
print(len(test_x_hypothesis_arr))
print(len(test_x_premise_arr))
print(len(test_y_arr))

1422
1422
1422


In [32]:
class CustomDataset(Dataset):
    def __init__(self, x_hypothesis_arr , x_premise_arr, y_list):
        self.samples = []
        while x_premise_arr:
            x_pre = x_premise_arr.pop()
            x_hyp = x_hypothesis_arr.pop()
            y = y_list.pop()
            x_pre_tensor = torch.tensor(x_pre,dtype = torch.float32)#.cuda()
            x_hyp_tensor = torch.tensor(x_hyp,dtype = torch.float32)#.cuda()
            y_tensor = torch.tensor(y,dtype = torch.float32)#.cuda()
            self.samples.append((x_pre_tensor,x_hyp_tensor,y_tensor))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

In [33]:
train_dataset = CustomDataset(train_x_hypothesis_arr , train_x_premise_arr, train_y_arr)
test_dataset = CustomDataset(test_x_hypothesis_arr , test_x_premise_arr, test_y_arr)

In [34]:
def get_maxlen(batch,i):
    maxlen = 0
    for tensr in batch:
        if tensr[i].shape[0] > maxlen :
           maxlen = tensr[i].shape[0]
    return maxlen

In [35]:
def collate_fn(batch):
    x_pres = []
    x_hyps = []
    ys = []
    pre_max_len = get_maxlen(batch,0)
    hyp_max_len = get_maxlen(batch,1)
    for tensr in batch:
        x_pre_tensor = tensr[0]
        x_hyp_tensor = tensr[1]
        y_tensor = tensr[2]
        pre_pad_width = pre_max_len - x_pre_tensor.shape[0]
        pre_leftpad_width = pre_pad_width // 2
        pre_rightpad_width = pre_pad_width - pre_leftpad_width
        pre_p1d = (0,0,pre_leftpad_width,pre_rightpad_width)
        paded_x_pre_tensor = F.pad(x_pre_tensor,pre_p1d,"constant",0)
        x_pres.append(paded_x_pre_tensor)
        hyp_pad_width = hyp_max_len - x_hyp_tensor.shape[0]
        hyp_leftpad_width = hyp_pad_width // 2
        hyp_rightpad_width = hyp_pad_width - hyp_leftpad_width
        hyp_p1d = (0,0,hyp_leftpad_width,hyp_rightpad_width)
        paded_x_hyp_tensor = F.pad(x_hyp_tensor,hyp_p1d,"constant",0)
        x_hyps.append(paded_x_hyp_tensor)
        ys.append(y_tensor)
    X_pre = torch.stack(x_pres,dim=0)
    X_hyp = torch.stack(x_hyps,dim=0)
    Y = torch.stack(ys,dim=0)
    return [X_pre,X_hyp,Y]

In [112]:
bsize = 128
train_loader = DataLoader(train_dataset, batch_size=bsize,collate_fn=collate_fn, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=bsize,collate_fn=collate_fn, shuffle=False)

In [113]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.lstm1 = nn.LSTM(100, 20, 1,batch_first = True)
        self.lstm2 = nn.LSTM(100, 20, 1,batch_first = True)
        self.fc1 = nn.Linear(20, 3)

    def forward(self,x_pre,x_hyp):
        #xpre_h0 = torch.zeros(1, bsize, 20)
        #xpre_c0 = torch.zeros(1, bsize, 20)
        output, (hn, cn) = self.lstm1(x_pre)
        xhyp_h0 = hn
        xhyp_c0 = cn
        output, (hn, cn) = self.lstm1(x_hyp, (xhyp_h0, xhyp_c0))
        a = self.fc1(hn)
        x = a.reshape([a.shape[1],a.shape[2]])
        x = nn.functional.softmax(x)
        return x

In [114]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = NN()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [116]:
model.train()
num_epochs = 10
for epoch in range(num_epochs):
    for x_pre, x_hyp,y in train_loader:
        optimizer.zero_grad()
        outputs = model(x_pre,x_hyp)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
    if loss.item() < 62 :
       torch.save(model,'model1.pth')
    print(loss.item())

print('Training complete.')

<ipython-input-113-5d8e21ec2923>:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = nn.functional.softmax(x)


0.7967789173126221
0.7489396333694458
0.8579851984977722
0.8289749026298523
1.028769850730896
0.8527803421020508
0.9194587469100952
0.741548478603363
0.7671436071395874
0.8246080279350281
Training complete.


In [123]:
model.eval()
predictions = []
true_labels = []
for x_pre, x_hyp, y in test_loader:
    outputs = model(x_pre,x_hyp)
    predicted_labels = torch.argmax(outputs, dim=1).cpu().numpy()
    predictions.extend(predicted_labels)
    l1 = torch.argmax(y, dim=1).cpu().numpy()
    true_labels.extend(l1)

accuracy = accuracy_score(true_labels, predictions)
print("Accuracy:", accuracy)

<ipython-input-113-5d8e21ec2923>:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = nn.functional.softmax(x)


Accuracy: 0.6547116736990155
